In [10]:
import os
import pickle
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from arabic_reshaper import reshape
from bidi.algorithm import get_display

from PIL import ImageFont, ImageDraw, Image
import pandas as pd

In [11]:
mp_holistic = mp.solutions.holistic 
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [12]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results
 

In [13]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
 

In [14]:
def extract_keypoints(results):
    data_aux = []

    x_ = []
    y_ = []

    if results.multi_hand_landmarks:
        
        if len(results.multi_hand_landmarks)==1:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

                for i in range(42, 84):
                    data_aux.append(0.0)

        if len(results.multi_hand_landmarks)==2:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_)) 
                
                
    else:
        for i in range(84):
            data_aux.append(0.0)                
    return data_aux  
 

In [15]:
actions = ['Act',
 'Alhamdullah',
 'all',
 'Baba',
 'Basis',
 'because',
 'Boy',
 'Brave',
 'calls',
 'calm',
 'College',
 'Confused',
 'conversation',
 'Crying',
 'daily',
 'danger',
 'disagree with',
 'drinks',
 'Eats',
 'effort',
 'Egypt',
 'Enters',
 'Excelent',
 'explanation',
 'Fasting',
 'Female',
 'First',
 'For Us',
 'fuel',
 'Gift',
 'Girl',
 'Glass',
 'good bye',
 'government',
 'Happy',
 'hates',
 'hears',
 'Help',
 'Here You Are',
 'how_are_u',
 'Humanity',
 'hungry',
 'I',
 'ignorance',
 'immediately',
 'Important',
 'Intelligent',
 'Last',
 'leader',
 'Liar',
 'Loves',
 'male',
 'Mama',
 'memory',
 'model',
 'mostly',
 'motive',
 'Muslim',
 'must',
 'my home land',
 'no',
 'nonsense',
 'obvious',
 'Old',
 'Palestine',
 'prevent',
 'ready',
 'rejection',
 'Right',
 'selects',
 'shut up',
 'Sing',
 'sleeps',
 'smells',
 'Somkes',
 'Spoon',
 'Summer',
 'symposium',
 'Tea',
 'teacher',
 'terrifying',
 'Thanks',
 'time',
 'to boycott',
 'to cheer',
 'to go',
 'to live',
 'to spread',
 'Toilet',
 'trap',
 'University',
 'ur_welcome',
 'victory',
 'walks',
 'Where',
 'wheres_ur_house',
 'Window',
 'Winter',
 'yes',
 'You']
 

In [16]:
reverb={'Act':'تتصرف',
      'Alhamdullah':'الحمدلله',
      'all':'جميعا',
      'Baba':'بابا',
      'Basis':'اساسيات',
      'because':'بسبب',
      'Boy':'ولد',
      'Brave':'شجاع',
      'calls':'اتصل',
     'calm':'هادئ',
     'College':'كليه',
     'Confused':'متوتر',
     'conversation':'محادثه',
     'Crying':'بكاء',
     'daily':'يوميا',
     'danger':'خطر',
     'disagree with':'اختلف مع',
     'drinks':'اشرب',
     'Eats':'اكل',
     'effort':'مجهود',
     'Egypt':'مصر',
     'Enters':'دخول', 
     'Excelent':'ممتاز',
     'explanation':'الشرح',
     'Fasting':'في الصيام',
     'Female':'انثي',
     'First':'اولا',
     'For Us':'لنا',
     'fuel':'الوقود',
     'Gift':'الهديه',
     'Girl':'البنت',
     'Glass':'الكوب',
     'good bye':'الي اللقاء',
     'government':'الحكومه',
     'Happy':'بسعاده',
     'hates':'اكره',
     'hears':'اسمع',
     'Help':'تساعد',
     'Here You Are':'تفضل',
     'how_are_u':'كيف حالك',
     'Humanity':'البشريه',
     'hungry':'اشعر بالجوع',
     'I':'انا',
     'ignorance':'تجاهل',
     'immediately':'حالا',
     'Important':'المهم',
     'Intelligent':'الذكاء',
     'Last':'الاخير',
     'leader':'القائد',
     'Liar':'الكاذب',
     'Loves':'احب',
     'male':'ذكر',
     'Mama':'ماما',
     'memory':'الذاكره',
     'model':'نموذج',
     'mostly':'في الغالب',
     'motive':'الدافع',
     'Muslim':'مسلم',
     'must':'لازم',
     'my home land':'وطني',
     'no':'لا',
     'nonsense':'الهراء',
     'obvious':'بديهي',
     'Old':'القديمه',
     'Palestine':'فلسطين',
     'prevent':'اوقف',
     'ready':'جاهز_ل',
     'rejection':'الرفض',
     'Right':'صحيح',
     'selects':'تختار',
     'shut up':'اصمت',
     'Sing':'الغني',
     'sleeps':'النوم',
     'smells':'اشم',
     'Somkes':'يسمح بالتدخين',
     'Spoon':'بالمعلقه',
     'Summer':'الصيف',
     'symposium':'الندوه',
     'Tea':'الشاي',
     'teacher':'معلمي',
     'terrifying':'مخيف',
     'Thanks':'شكرا لكم',
     'time':'الوقت',
     'to boycott':'تقاطع لاجل',
     'to cheer':'تهتف ل',
     'to go':'ذاهب الي',
     'to live':'ل اعيش',
     'to spread':'تنتشر',
     'Toilet':'دوره المياه',
     'trap':'فخ',
     'University':'الجامعه',
     'ur_welcome':'مرحباً بكم',
     'victory':'النصر',
     'walks':'المشي',
     'Where':'اين',
     'wheres_ur_house':'اين تسكن',
     'Window':'الشباك',
     'Winter':'الشتاء',
     'yes':'نعم',
     'You':'انت'}

In [17]:
from keras.models import load_model

# Path to your saved model file (.h5 format)

# Load the model
modelLSTM =  load_model("action3.h5")

In [19]:
# 1. New detection variables
import datetime
import time

sequence = []
sentence = []
threshold = 0.99

hands = mp_hands.Hands(min_tracking_confidence=0.7, min_detection_confidence=0.7 , max_num_hands = 2, static_image_mode=True )

cap = cv2.VideoCapture(0)

timeout_minutes = 0.5 
last_update_time = datetime.datetime.now()                            

while cap.isOpened():

    # Read feed
    ret, frame = cap.read()
    
    frame = cv2.flip(frame,1)
    # Make detections
    image, results = mediapipe_detection(frame, hands)
    

      

    if results.multi_hand_landmarks:
        keypoints = extract_keypoints(results)
        if len(keypoints)== 84:
            sequence.append(keypoints)
            if len(sequence) == 15 :
                res = modelLSTM.predict(np.expand_dims(sequence, axis=0))[0]
                EN = actions[np.argmax(res)]
                print(reverb[EN])
                print(res[np.argmax(res)])
                sequence = []




        #3. Viz logic
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if reverb[EN] != sentence[-1]:
                            sentence.append(reverb[EN])
                            last_update_time = datetime.datetime.now()                            
                    else:
                        sentence.append(reverb[EN])
                        last_update_time = datetime.datetime.now()
                                      
    else:
        sequence = []
        
        
    current_time = datetime.datetime.now()
    time_elapsed_minutes = (current_time - last_update_time).total_seconds() / 60

    if time_elapsed_minutes > timeout_minutes and len(sentence) > 0:
        sentence.clear()

            

    fontpath = "Arial.ttf" # <== download font
    font = ImageFont.truetype(fontpath, 32)
    text_color = (0, 255, 255)
    upper_part_height = 40
    reshaped_sentence = [get_display(reshape(x)) for x in sentence[::-1]]
    img_pil = Image.fromarray(frame)
    draw = ImageDraw.Draw(img_pil)
    draw.rectangle([(0, 0), (frame.shape[1], upper_part_height)], fill=(255, 0, 0))  
    draw.text((0,0), ' '.join(reshaped_sentence), font=font, fill=text_color)
    img = np.array(img_pil)
    draw_styled_landmarks(img, results)

    
    cv2.imshow('Frame', img)
    # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
 

1/1 [==============================] - 0s 37ms/step
الي اللقاء
0.9991129
